In [1]:
import pandas as pd

In [ ]:
# 

In [15]:
"""Using this notebook, I will add """

# Change the genes!!! 
genes = []
all_revel_files = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\REVEL\\all_REVEL_predictions.csv")

for gene in genes:
    new_gene_path = f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\REVEL\\{gene}_revel_with_pos.csv"
    # read the file of the new gene
    new_gene_predictions = pd.read_csv(new_gene_path)
    # add "gene" column to the dataframe
    new_gene_predictions["gene"] = gene
     
    # add the new gene to the all_eve_files dataframe, without the header
    all_revel_files = pd.concat([all_revel_files, new_gene_predictions], ignore_index=True, axis=0)

# Save the new dataframe to a csv file
all_revel_files.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\REVEL\\all_REVEL_predictions.csv", index=False)

In [1]:
# Combine the prediction files into one file
my_prediction = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\predictions_vs_real_with_variant_all_genes_updated_221123.csv")
my_prediction

NameError: name 'pd' is not defined

In [ ]:
REVEL_predictions = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\REVEL\\all_REVEL_predictions.csv")
# Remove rows with gene null
REVEL_predictions = REVEL_predictions[REVEL_predictions["gene"].notnull()]
REVEL_predictions

In [ ]:
# Only leave "wt_aa", "position" "mt_aa" and "EVE_scores_ASM" columns
REVEL_predictions = REVEL_predictions[['aa_wt', 'aa_mut', 'aa_pos', 'revel_score', 'gene']]
REVEL_predictions["aa_pos"] = REVEL_predictions["aa_pos"].astype(str)
# create new row "variant"
REVEL_predictions['variant'] = REVEL_predictions['aa_wt'] + REVEL_predictions['aa_pos'] + REVEL_predictions['aa_mut']
REVEL_predictions = REVEL_predictions.drop(columns=["aa_wt", "aa_mut", "aa_pos"])
REVEL_predictions

In [ ]:
print(f"length of revel including VUS {len(REVEL_predictions)}")

In [ ]:
# Remove VUS variants
REVEL_predictions["revel_score"] = pd.to_numeric(REVEL_predictions['revel_score'], errors='coerce')
# Remove rows with revel score: 0.29 < revel score < 0.644
REVEL_VUS = REVEL_predictions[(REVEL_predictions["revel_score"] < 0.644) & (REVEL_predictions["revel_score"] >= 0.29)]
REVEL_predictions = REVEL_predictions[(REVEL_predictions["revel_score"] >= 0.644) | (REVEL_predictions["revel_score"] <= 0.29)]
REVEL_predictions

In [ ]:
# Add benign or pathogenic label to each variant, according to the VEST4 score
REVEL_predictions["REVEL_pathogenicity"] = REVEL_predictions["revel_score"].apply(lambda x: 1 if x >= 0.644 else 0)
REVEL_predictions

In [ ]:
# Save REVEL_predictions to csv
REVEL_predictions.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\REVEL\\REVEL_predictions_no_VUS.csv")

In [ ]:
print(f"length of revel without VUS {len(REVEL_predictions)}")
print(f"length of revel VUS {len(REVEL_VUS)}")

In [ ]:
# Merge the two files
merged = pd.merge(my_prediction, REVEL_predictions, on=["gene", "variant"])
merged

In [ ]:
# To make sure that all tools will be tested on the same variants, save the merged with REVEL file and do so for all other tools. Then merge all the files together.
merged_only_REVEL_columns = merged[['gene', 'variant', 'revel_score', 'REVEL_pathogenicity']]
merged_only_REVEL_columns.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\REVEL_on_dvd_data_predictions_LOPO.csv")

In [63]:
####################################################################################################################################################################

In [64]:
print(f"length of merged {len(merged)}")
print(f"length of my_prediction {len(my_prediction)}")
print(f"length of REVEL {len(REVEL_predictions)}")

length of merged 3016
length of my_prediction 4521
length of REVEL 36574


In [ ]:
############################################################################################################
# All next rows were used to calculate the MCC, before realised I must apply first the threshold and remove the VUS variants. Now all these rows aren't in use, since I'm doing the calculations after removing the VUS variants and with all tools together.#############################################################################################################

In [ ]:
# ## Calculate the MCC for the mutpred predictions, according to the real pathogenicity (0 or 1)
# from sklearn.metrics import matthews_corrcoef
# 
# # separate the merged dataframe according to gene
# genes = merged["gene"].unique()
# print(f"Number of genes: {len(genes)}")
# # Calculate MCC for each gene specific predictor for EVE
# 
# ### REVEL ####
# 
# # Build dictionary with gene names as keys.
# mccs = {gene: 0 for gene in genes}
# 
# for gene in genes:
#     gene_df = merged[merged["gene"] == gene]
#     gene_df['revel_score'] = gene_df['revel_score'].astype(float)
#     # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
#     # Create binary predictions based on the 0.5 threshold
#     gene_df.loc[:, 'binary_prediction_REVEL'] = (gene_df['revel_score'] > 0.644).astype(int)
#     
#     gene_df['predictions'] = gene_df['predictions'].astype(int)
#     
#     # Calculate MCC
#     mcc = matthews_corrcoef(gene_df['predictions'], gene_df['binary_prediction_REVEL'])
#     
#     # Get gene name for the use for the dictionary
#     gene = gene_df['gene'].unique()[0]
#     
#     # Append mcc to dictionary
#     mccs[gene] = mcc
# 
# print("MCCs of REVEL predictions for each gene:")
# for gene in mccs:
#     print(f"{gene}: {mccs[gene]}")

In [ ]:

# from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score, \
#     log_loss, confusion_matrix
# 
# # store the results in a dictionary
# results = {gene: 0 for gene in genes}
# 
# for gene in genes:
#     gene_df = merged[merged["gene"] == gene]
#     gene_df['revel_score'] = gene_df['revel_score'].astype(float)
#     y_true = gene_df["pathogenicity"].astype(int)
#     predicted_probabilities = gene_df["revel_score"]
#     gene_df.loc[:, 'binary_prediction_REVEL'] = (gene_df['revel_score'] > 0.644).astype(int)
#     predicted_labels = gene_df["binary_prediction_REVEL"]
# 
#     results_gene = {}
#     # Calculate AUC-ROC
#     roc_auc = roc_auc_score(y_true, predicted_probabilities)
#     # append to dictionary
#     results_gene["roc_auc"] = roc_auc
# 
#     # Calculate ROC curve
#     fpr, tpr, thresholds = roc_curve(y_true, predicted_probabilities)
#     roc_auc = auc(fpr, tpr)
#     # append to dictionary
#     results_gene["auc"] = roc_auc
# 
#     # Calculate AUC-PR
#     precision, recall, _ = precision_recall_curve(y_true, predicted_probabilities)
#     pr_auc = auc(recall, precision)
#     # append to dictionary
#     results_gene["pr_auc"] = pr_auc
# 
#     # Calculate F1 Score
#     f1 = f1_score(y_true, predicted_labels)
#     # append to dictionary
#     results_gene["f1"] = f1
# 
#     # Calculate Log Loss
#     logloss = log_loss(y_true, predicted_probabilities)
#     # append to dictionary
#     results_gene["logloss"] = logloss
# 
#     # Calculate confusion matrix
#     conf_matrix = confusion_matrix(y_true, predicted_labels)
#     # append to dictionary
#     results_gene["confusion_matrix"] = conf_matrix
# 
#     # Get gene name for the use for the dictionary
#     gene = gene_df['gene'].unique()[0]
#     # Append results to dictionary
#     results[gene] = results_gene
# 
# ## Print results
# print("Results of REVEL predictions for each gene:")
# for gene in results:
#     print(f"{gene}: {results[gene]}")

In [ ]:
# # present the confusion matrix for each gene
# import matplotlib.pyplot as plt
# 
# for gene in results:
#     # Plot confusion matrix
#     plt.imshow(results[gene]["confusion_matrix"], cmap=plt.cm.Blues)
#     plt.xlabel("Predicted labels")
#     plt.ylabel("True labels")
#     plt.xticks([0, 1], ["Benign", "Pathogenic"])
#     plt.yticks([0, 1], ["Benign", "Pathogenic"])
#     plt.title(f"Confusion matrix for {gene}, REVEL predictions")
#     plt.style.use("seaborn-white")
#     # Add text annotations
#     for i in range(2):
#         for j in range(2):
#             plt.text(j, i, results[gene]["confusion_matrix"][i, j], ha="center", va="center", color="black",
#                      backgroundcolor="white")
#     plt.colorbar()
#     plt.show()

In [ ]:
# # present the ROC curve for each gene
# for gene in results:
#     # Plot ROC curve
#     plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {results[gene]['roc_auc']:.2f})")
#     plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel("False Positive Rate")
#     plt.ylabel("True Positive Rate")
#     plt.title(f"ROC curve for {gene}, REVEL predictions")
#     plt.legend(loc="lower right")
#     plt.show()